In [2]:
# vector_lab.ipynb
import os
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# ---------- CONFIG ----------
os.environ["OPENAI_API_BASE"] = "http://localhost:1234/v1"   # LM Studio endpoint
os.environ["OPENAI_API_KEY"] = "lm-studio"

PDF_PATH = "English_textbook_11th.pdf"
PICKLE_FILE = "vectorstore.pkl"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
K_NEIGHBORS = 3

# models
EMBED_MODEL_PRIORITY = [
    "text-embedding-nomic-embed-text-v1.5",
    "sentence-transformers/all-mpnet-base-v2"
]
LLM_PRIORITY = ["phi-2", "tinyllama-1.1b-chat-v1.0"]
# ----------------------------

# ---------- LOADING PDF ----------
print("🔄 Loading PDF...")
loader = PDFPlumberLoader(PDF_PATH)
docs = loader.load()
print(f"✅ Loaded {len(docs)} pages from {PDF_PATH}")

# ---------- SPLITTING ----------
print("✂️  Splitting into chunks...")
splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
documents = splitter.split_documents(docs)
print(f"✅ Split into {len(documents)} chunks")

# ---------- EMBEDDINGS ----------
def load_embedder():
    for model_name in EMBED_MODEL_PRIORITY:
        try:
            print(f"🧠 Loading embedding model: {model_name}")
            if model_name.startswith("text-embedding-nomic"):
                return OpenAIEmbeddings(
                    model=model_name,
                    openai_api_base=os.environ["OPENAI_API_BASE"],
                    openai_api_key=os.environ["OPENAI_API_KEY"]
                )
            else:
                return HuggingFaceEmbeddings(model_name=model_name)
        except Exception as e:
            print(f"⚠️ Failed to load {model_name}: {e}")
    raise RuntimeError("❌ No embedding model could be initialized.")


embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

print("🧠 Creating embeddings (this may take a while)...")
vectorstore = FAISS.from_documents(documents, embedder)
print("✅ FAISS vectorstore created")

# 💾 Save vectorstore
with open(PICKLE_FILE, "wb") as f:
    pickle.dump(vectorstore, f)
print(f"💾 Vectorstore saved to {PICKLE_FILE}")

# ---------- PROMPT ----------
prompt_template = """
You are a knowledgeable assistant.
Use only the provided context to answer the question.
If the answer isn't found, respond:
"The information is not available in the provided context."

Context:
{context}

Question:
{input}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_template)

# ---------- MODEL HANDLER ----------
def try_model(model_name):
    print(f"\n🚀 Trying model: {model_name}")
    try:
        llm = ChatOpenAI(
            model=model_name,
            temperature=0.6,
            openai_api_base=os.environ["OPENAI_API_BASE"],
            openai_api_key=os.environ["OPENAI_API_KEY"],
            request_timeout=300,
            streaming=True
        )
        combine_chain = create_stuff_documents_chain(llm, QA_CHAIN_PROMPT)
        retriever = vectorstore.as_retriever(search_kwargs={"k": K_NEIGHBORS})
        return create_retrieval_chain(retriever, combine_chain)
    except Exception as e:
        print(f"⚠️ Model {model_name} failed: {e}")
        return None

qa_chain = None
for model in LLM_PRIORITY:
    qa_chain = try_model(model)
    if qa_chain:
        break

if not qa_chain:
    print("❌ All models failed to load. Try running LM Studio with one of the supported models (phi-2 or tinyllama).")
else:
    # ---------- EVALUATION ----------
    print("\n🧪 Running Evaluation Mode...")

    eval_set = [
        {"question": "What is the letter to the son about?",
         "answer": "It is a letter from a father giving advice and moral guidance to his son."},
        {"question": "Who wrote the poem 'Commonwealth of Bees'?",
         "answer": "William Shakespeare"},
        {"question": "What is the theme of the lesson 'Will He Come Home?'?",
         "answer": "The story shows a mother's anxiety when her son is late returning home."}
    ]

    total_cosine, correct_count = 0, 0
    total = len(eval_set)

    for i, sample in enumerate(eval_set, 1):
        q, expected = sample["question"], sample["answer"]
        print(f"\n🔹 [{i}] Q: {q}")
        result = qa_chain.invoke({"input": q})
        predicted = result.get("answer") or result.get("result") or "No answer"

        print(f"🤖 Model: {predicted}")
        print(f"🎯 Expected: {expected}")

        emb_pred = embedder.embed_query(predicted)
        emb_true = embedder.embed_query(expected)
        sim = cosine_similarity([emb_pred], [emb_true])[0][0]
        total_cosine += sim

        if expected.lower().split()[0] in predicted.lower():
            correct_count += 1

        print(f"🔸 Cosine Similarity: {sim:.3f}")

    avg_cosine = total_cosine / total
    accuracy = correct_count / total

    print("\n📊 --- Evaluation Summary ---")
    print(f"✅ Accuracy (rough match): {accuracy * 100:.1f}%")
    print(f"🧩 Avg Semantic Similarity: {avg_cosine:.3f}")
    print("-----------------------------")


🔄 Loading PDF...
✅ Loaded 195 pages from English_textbook_11th.pdf
✂️  Splitting into chunks...
✅ Split into 459 chunks
🧠 Creating embeddings (this may take a while)...
✅ FAISS vectorstore created
💾 Vectorstore saved to vectorstore.pkl

🚀 Trying model: phi-2

🧪 Running Evaluation Mode...

🔹 [1] Q: What is the letter to the son about?
🤖 Model: 
The letter expresses Abraham Lincoln’s concern for his son who is starting school today. He asks the teacher to treat the child gently as he will have a new and strange experience in school. The father wants the teacher to be kind, understanding, and patient with the child. He also emphasizes that children are naturally afraid of going to school on their first day. Lincoln believes that this fear is due to being away from home for the first time and not knowing what to expect. 
The letter contains some general principles of education that can help in shaping a child’s personality, such as gracefully losing and winning, having faith in one’s ideas